In [ ]:
import csv
import matplotlib as plot
import numpy as np
import random
import re
import os

In [ ]:
def vbias_filter(input_file, output_file):
    """extract vbias value from netlist to .txt file"""
    filtered_lines = []
    with open(input_file, "r") as infile:
        for line in infile:
            stripped_line = line.lstrip()  # Remove leading whitespace
            words = line.split()
            if (stripped_line.startswith("vbias") and len(words) > 1 ):
                filtered_lines.append(line.strip())  # Store non-empty lines without leading/trailing spaces
    with open(output_file, "w") as outfile:
        outfile.write("\n".join(filtered_lines))  # Write all lines at once   

def transistor_size_filter(input_file, output_file):
    """extract transistor size from netlist to .txt file"""
    filtered_lines = []
    with open(input_file, "r") as infile:
        for line in infile:
            stripped_line = line.lstrip()  # Remove leading whitespace
            words = line.split()
            if (stripped_line.startswith("M") and len(words) > 1 ):
                filtered_lines.append(line.strip())  # Store non-empty lines without leading/trailing spaces
    with open(output_file, "w") as outfile:
        outfile.write("\n".join(filtered_lines))  # Write all lines at once   

def convert_to_csv_vbias(input_txt, output_csv):
    """convert .txt to .csv for bias"""
    headers = []
    values = []

    with open(input_txt, "r") as infile:
        for line in infile:
            stripped_line = line.strip()

            # If the line starts with 'device', extract the device names (excluding the word 'device')
            if stripped_line.startswith("vbias"):
                vbias = stripped_line.split()[1]  # Skip the word 'device'
                headers.append(vbias)
                value = stripped_line.split()[-1]
                values.append(value)

                num_columns = len(headers)
                values_2d = [values[i:i+num_columns] for i in range(0, len(values), num_columns)]
                with open(output_csv, "w", newline="") as outfile:
                    csv_writer = csv.writer(outfile)
        # Write the header row (device names)
                    csv_writer.writerow(headers)
        # Write the gm values (rows)
                    csv_writer.writerows(values_2d)  

def remove_units(value):
    """Remove 'u', 'n', 'p', 'm', etc. from transistor sizes and keep only the numeric values."""
    return re.sub(r'[a-zA-Z]', '', value)

def convert_to_csv_size(input_txt, output_csv):
    """convert .txt to .csv for size"""
    headers = []
    widths = []
    lengths = []

    with open(input_txt, "r") as infile:
        for line in infile:
            stripped_line = line.strip()

        # If the line starts with 'gm', extract the gm values
            if stripped_line.startswith("M"):
                device = stripped_line.split()[0] 
                headers.append(device)
                words = line.split()

                for word in words:
                    if word.startswith("W="):
                        width = word[2:]  # Store the next word
                        widths.append(remove_units(width))

                    elif word.startswith("L="):
                        length = word[2:]  # Store the next word
                        lengths.append(remove_units(length))

                num_columns = len(headers)
                width_2d = [widths[i:i+num_columns] for i in range(0, len(widths), num_columns)]
                length_2d = [lengths[i:i+num_columns] for i in range(0, len(lengths), num_columns)]

                #print(width_2d)

                with open(output_csv, "w", newline="") as outfile:
                    csv_writer = csv.writer(outfile)
        # Write the header row (device names)
                    csv_writer.writerow(headers)
        # Write the gm values (rows)
                    csv_writer.writerows(width_2d)  
                    csv_writer.writerows(length_2d)  

In [ ]:
def process_circuit_file(input_cir_file):
    """
    Processes a circuit file by extracting bias and size information, converting it to CSV format.

    Parameters:
        input_cir_file (str): Path to the input circuit file (e.g., 'a5').
    """
    # Step 1: Extract bias information and save to text file
    vbias_filter(input_cir_file, 'output/vbias.txt')
    
    # Step 2: Extract transistor size information and save to text file
    transistor_size_filter(input_cir_file, 'output/size.txt')
    
    # Step 3: Convert bias text file to CSV
    convert_to_csv_vbias('output/vbias.txt', 'output/vbias.csv')
    
    # Step 4: Convert size text file to CSV
    convert_to_csv_size('output/size.txt', 'output/size.csv')

    print(f"Processing complete.")

#can change the circuit file
cir_file = 'a5.cir'

process_circuit_file(cir_file)

In [ ]:
def apply_variation(value, miu, sigma):
    """
    Applies a random variation 
    """
    variation = random.gauss(miu, sigma)  # Random variation between -10% and +10%
    print(variation)
    return value * (1 + variation)

def process_csv(input_file, output_file, miu, sigma):
    """
    Reads the input CSV file, applies random variation to all numeric values,
    and writes the results to the output CSV file.
    """
    with open(input_file, "r") as infile, open(output_file, "w", newline="") as outfile:
        csv_reader = csv.reader(infile)
        csv_writer = csv.writer(outfile)

        for row in csv_reader:
            # Process each cell in the row
            new_row = []
            for cell in row:
                try:
                    # Try to convert the cell to a float
                    numeric_value = float(cell)
                    # Apply random variation
                    new_value = apply_variation(numeric_value, miu, sigma )
                    # Round to 2 decimal places (optional)
                    new_value = round(new_value, 2)
                    new_row.append(str(new_value))
                except ValueError:
                    # If the cell is not a number, keep it unchanged
                    new_row.append(cell)
            # Write the modified row to the output file
            csv_writer.writerow(new_row)


def replace_bias_values(text_file, csv_file, output_file):
    """
    Replaces bias values in a text file using data from a CSV file.

    :param text_file: Path to the text file.
    :param csv_file: Path to the CSV file.
    :param output_file: Path to the output file.
    """
    # Step 1: Read the CSV file and store the new bias values
    new_bias_values = {}
    with open(csv_file, "r") as csvfile:
        csv_reader = csv.reader(csvfile)
        headers = next(csv_reader)  # Read the header row (bias1, bias2, etc.)
        values = next(csv_reader)  # Read the values row (0.94, 1.1, etc.)
        for header, value in zip(headers, values):
            new_bias_values[header.strip()] = value.strip()

    # Step 2: Read the text file and replace bias values
    with open(text_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            # Split the line into components
            parts = line.split()
            if len(parts) >= 5 and parts[0].startswith("vbias"):  # Check if the line starts with "vbias"
                bias_header = parts[1]  # Extract the bias header (e.g., bias1, bias2)
                if bias_header in new_bias_values:
                    # Replace the bias value with the new value from the CSV file
                    parts[4] = new_bias_values[bias_header]  # Update the DC value
                    line = " ".join(parts) + "\n"  # Reconstruct the line
            # Write the updated line to the output file
            outfile.write(line)

def replace_size_values(text_file, csv_file, output_file):
    """
    Replaces transistor width (W) and length (L) numerical values in a text file using data from a CSV file.

    :param text_file: Path to the input SPICE netlist file.
    :param csv_file: Path to the CSV file containing updated W and L values.
    :param output_file: Path to the output file with updated values.
    """
    # Step 1: Read the CSV file and store new width (W) and length (L) values
    new_width_values = {}
    new_length_values = {}

    with open(csv_file, "r") as csvfile:
        csv_reader = csv.reader(csvfile)
        headers = next(csv_reader)  # Read the header row (M3, M4, M6, etc.)
        widths = next(csv_reader)   # Read the W values row
        lengths = next(csv_reader)  # Read the L values row

        for header, width, length in zip(headers, widths, lengths):
            new_width_values[header.strip()] = f"{width.strip()}u"  # Append 'u' for width
            new_length_values[header.strip()] = f"{length.strip()}n"  # Append 'n' for length

    # Step 2: Read the text file and replace only W and L values
    with open(text_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            parts = line.split()

            if parts and parts[0] in new_width_values:  # Check if the transistor is in the CSV data
                device_name = parts[0]

                # Replace only the numeric values of W and L
                new_line = []
                for part in parts:
                    if part.startswith("W="):
                        new_line.append(f"W={new_width_values[device_name]}")
                    elif part.startswith("L="):
                        new_line.append(f"L={new_length_values[device_name]}")
                    else:
                        new_line.append(part)

                outfile.write(" ".join(new_line) + "\n")
            else:
                outfile.write(line)  # Write unchanged lines

In [ ]:

def monte_carlo(input_cir, number):
    input_csv_bias = 'output/vbias.csv'
    input_csv_size = 'output/size.csv'
    output_csv_bias = "output/bias_var.csv"
    output_csv_size = "output/size_var.csv"
    for i in range(number):
        ''' miu, sigma can be changed'''
        process_csv(input_csv_bias, output_csv_bias, 0, 0.1)
        process_csv(input_csv_size, output_csv_size, 0, 0.0001)
        output_file = f"output/var{i + 1}.cir"
        replace_bias_values(input_cir, output_csv_bias, f"output/size_var{i + 1}.cir")
        replace_size_values(f"output/size_var{i + 1}.cir", output_csv_size, output_file)
        if os.path.exists(f"output/size_var{i + 1}.cir"):
            os.remove(f"output/size_var{i + 1}.cir")

In [ ]:
monte_carlo(input_cir = 'a5.cir', number = 10)

In [ ]:
def vbias_variation(number):
# Example usage
    input_csv = "a5_vbias.csv"  # Replace with your input CSV file
    output_csv = "a5_vbias_var.csv"  # Replace with your desired output CSV file
    
    text_file = "a5.cir"  # Replace with your text file
    csv_file = "a5_vbias_var.csv"  # Replace with your CSV file
    for i in range(number):
        process_csv(input_csv, output_csv)
        output_file = f"a5_bias_var{i}.cir"  # Replace with your desired output file
        # Process the files
        replace_bias_values(text_file, csv_file, output_file)
        print('bias updated')
        print(f"Updated file saved to {output_file}")
        #size_variation(1, 'a5_bias_var{i}.cir')



def size_variation(number, text_file):
# Example usage
    input_csv = "a5_size.csv"  # Replace with your input CSV file
    output_csv = "a5_size_var.csv"  # Replace with your desired output CSV file
    
    #text_file = "a5.cir"  # Replace with your text file
    csv_file = "a5_size_var.csv"  # Replace with your CSV file
    for i in range(number):
        process_csv(input_csv, output_csv)
        output_file = f"a5_size_var{i}.cir"  # Replace with your desired output file
        # Process the files
        replace_size_values(text_file, csv_file, output_file)
        print(f"Updated file saved to {output_file}")

